In [ ]:
## basic imports for collecting files

import os
import time
import uuid 
import cv2

In [ ]:
# Images path
images_path = os.path.join(r"D:\PROJECTS\Opencv projects\cus_obj_detection\data")
number_images = 30


# capture images
cap = cv2.VideoCapture(0)

for imgnum in range(number_images):
    print("collecting images{}" . format(imgnum))
    
    ret , frame = cap.read()
    imgname = os.path.join(images_path , f"{str(uuid.uuid1())}.jpg")
    cv2.imwrite(imgname, frame)
    cv2.imshow("frame" , frame)
    time.sleep(0.5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# script for splitting the data
import random
import shutil
import os

# paths 
imgs_dir = r"D:\PROJECTS\Opencv projects\cus_obj_detection\New Data\images"
labels_dir = r"D:\PROJECTS\Opencv projects\cus_obj_detection\New Data\labels"

# output structure
output = "dataset"
train_img_dir = os.path.join(output, "images/train")
val_img_dir = os.path.join(output, "images/val")
train_lbl_dir = os.path.join(output, "labels/train")
val_lbl_dir = os.path.join(output, "labels/val")

# create folders
for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# get all image files
img_files = [f for f in os.listdir(imgs_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

# shuffle and split (80% train, 20% val)
random.seed(42)
random.shuffle(img_files)
split_idx = int(0.8 * len(img_files))
train_files = img_files[:split_idx]
val_files = img_files[split_idx:]

# helper function
def move_files(file_list, img_out, lbl_out):
    for file in file_list:
        img_path = os.path.join(imgs_dir, file)
        lbl_path = os.path.join(labels_dir, os.path.splitext(file)[0] + ".txt")

        # copy image
        shutil.copy(img_path, os.path.join(img_out, file))

        # copy label if it exists
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(lbl_out, os.path.splitext(file)[0] + ".txt"))
        else:
            print(f"⚠️ No label found for {file}")

# move train and val files
move_files(train_files, train_img_dir, train_lbl_dir)
move_files(val_files, val_img_dir, val_lbl_dir)

print(f"{len(train_files)} training and {len(val_files)} validation images created")

In [ ]:
#import and train the YOLOV8 model


from ultralytics import YOLO

model  = YOLO("yolov8n.pt")

# train

model.train(
    data="D:\PROJECTS\Opencv projects\cus_obj_detection\config.yaml",
    epochs=100,
    imgsz=640,
    batch=16
)

In [ ]:
# check the metrics of the trained model
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")
metrics = model.val()
print(metrics)

In [ ]:
# to use the model trained on best weights
!cp runs/detect/train2/weights/best.pt /content/drive/MyDrive/best_yolo.pt

In [ ]:
# GIVE PREDICTION ON 10 RANDOM IMAGES TO CHECK THE ACCURACY OF THE MODEL

def predict_random_images_gpu(model_path, images_folder, output_folder="predictions", top_n=10):
    # Load your trained YOLOv8 model
    model = YOLO(model_path)

    # Make sure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get all image paths
    all_images = [os.path.join(images_folder, f) for f in os.listdir(images_folder)
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Pick 10 random images (or fewer if not enough images)
    random_images = random.sample(all_images, min(10, len(all_images)))

    for image_path in random_images:
        print(f"\nProcessing: {image_path}")

        # Predict using GPU explicitly
        results = model.predict(image_path, conf=0.5, imgsz=640, device='cuda:0')

        # Access boxes, scores, and class names
        boxes = results[0].boxes
        scores = boxes.conf.cpu().numpy()
        classes = boxes.cls.cpu().numpy()
        names = results[0].names

        # Pick top `top_n` predictions
        top_indices = np.argsort(scores)[::-1][:top_n]

        # Print top predictions
        print("Top predictions:")
        for i in top_indices:
            cls_id = int(classes[i])
            print(f"Class: {names[cls_id]}, Confidence: {scores[i]:.2f}, Box: {boxes.xyxy[i].cpu().numpy()}")

        # Draw top predictions on the image
        img = cv2.imread(image_path)
        for i in top_indices:
            x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy().astype(int)
            cls_id = int(classes[i])
            label = f"{names[cls_id]} {scores[i]:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        # Save annotated image
        out_path = os.path.join(output_folder, os.path.basename(image_path))
        cv2.imwrite(out_path, img)
        print(f"Saved annotated image to: {out_path}")

    print(f"\nDone! Annotated images saved in folder: {output_folder}")


# -----------------------------
# USAGE
# -----------------------------
model_path = r"D:\PROJECTS\Opencv projects\cus_obj_detection\obj.pt"

# Folder containing images to predict on
images_folder = r"D:\PROJECTS\Opencv projects\cus_obj_detection\prediction"

# Run prediction
predict_random_images_gpu(model_path, images_folder)


In [ ]:
# LIVE TESTING OF THE CUSTOM OBJECT DETECTION MODEL

# Load your trained YOLOv8 model
model = YOLO("obj.pt")  # replace with your actual path

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection (uses GPU automatically)
    results = model.predict(frame, conf=0.5)

    # Draw bounding boxes
    annotated_frame = results[0].plot()

    # Display live video
    cv2.imshow("YOLOv8 Live Detection", annotated_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
